In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense
from tensorflow.keras.optimizers import Adam
#import keras_tuner as kt

#file_path = 'dataset.csv'
df = pd.read_csv(file_path)

df['date'] = pd.to_datetime(df['date'], format='%m/%d/%Y')
df = df.sort_values('date').reset_index(drop=True)

encoder = LabelEncoder()
df['news_sentiment'] = encoder.fit_transform(df['news_sentiment'])
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df[['news_sentiment', 'opening price', 'high', 'low', 'volume traded', 'closing price']])

sequence_length = 20
X = []
y = []

for i in range(sequence_length, len(scaled_data)):
    X.append(scaled_data[i-sequence_length:i])
    y.append(scaled_data[i, -1])
X, y = np.array(X), np.array(y)
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

def build_bilstm_model(hp):
    model = Sequential()

    model.add(Bidirectional(LSTM(units=hp.Int('units_1', min_value=32, max_value=128, step=32), return_sequences=True), input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Bidirectional(LSTM(units=hp.Int('units_2', min_value=32, max_value=128, step=32))))

    model.add(Dense(1))

    model.compile(optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
                  loss='mean_squared_error')
    return model

tuner = kt.RandomSearch(
    build_bilstm_model,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    directory='keras_tuner_dir',
    project_name='bilstm_stock_price_prediction'
)

tuner.search(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

best_hps = tuner.get_best_hyperparameters()[0]
print(f"Best Hyperparameters: {best_hps.values}")

best_model = tuner.hypermodel.build(best_hps)
best_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

bilstm_predictions = best_model.predict(X_test).flatten()

bilstm_predictions_inverse = scaler.inverse_transform(np.column_stack([np.zeros((len(bilstm_predictions), 5)), bilstm_predictions]))[:, -1]
real_prices = scaler.inverse_transform(np.column_stack([np.zeros((len(y_test), 5)), y_test]))[:, -1]

mse = mean_squared_error(real_prices, bilstm_predictions_inverse)
mae = mean_absolute_error(real_prices, bilstm_predictions_inverse)
r2 = r2_score(real_prices, bilstm_predictions_inverse)
mape = np.mean(np.abs((real_prices - bilstm_predictions_inverse) / real_prices)) * 100
correct_direction = np.sign(np.diff(real_prices)) == np.sign(np.diff(bilstm_predictions_inverse))
pocid = np.mean(correct_direction) * 100

print(f'Mean Squared Error (BiLSTM): {mse}')
print(f'Mean Absolute Error (BiLSTM): {mae}')
print(f'R2 score (BiLSTM): {r2}')
print(f'Mean Absolute Percentage Error (MAPE): {mape}%')
print(f'Percentage of Correct Directional Change (POCID): {pocid}%')

test_dates = df['date'][-len(real_prices):]
plt.figure(figsize=(25, 6))
plt.plot(test_dates, real_prices, color='black', label='Actual TSLA Closing Price')
plt.plot(test_dates, bilstm_predictions_inverse, color='blue', linestyle='dashed', label='BiLSTM Predictions')
plt.title('TSLA Stock Price Prediction (BiLSTM) with Keras Tuner Hyperparameter Tuning')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.xticks(rotation=45)
plt.show()
